### Method 1

In [3]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
import pandas as pd
from collections import defaultdict
from libindic import inexactsearch
sys.path.append('../method2')
from People_translator import Translation_Api
translator = Translation_Api()

In [4]:
properties = {
    "name": "P2561",
    "native_name": "P1559",
    "image": "P18",
    "flag": "P163",
    "map": "P242",
    "motto":"P1546",
    "timezone": "P2907",
    "country": "P17",
    "location": "P276",
    "area / area_km2": "P2046",
    "length / length_km": "P2043",
    "width / width_km": "P2049",
    "population": "P1082",
    "elevation / elevation_m":"P2044" ,
    "geology" : "P2695",
}

to_transliteration = ['motto']

In [5]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

endpoint_url = "https://query.wikidata.org/sparql"

In [15]:
#calls qwikidata get entity function for an entity id
def getEntityInfo(eid):
    return get_entity_dict_from_api(eid)

#extract the name of the entity or the property value in native language
def extractName(info):
    return info.get('labels', {}).get('en', {}).get('value', "")

#extract the description of the entity or the property value in native language
def extractDescription(info):
    return info.get('descriptions', {}).get('en', {}).get('value', "")
    
def method1_infobox(wd, bio):
    result = {}
    #get entity from api
    entity_info = getEntityInfo(wd)
    result['name'] = translator.get_transliteration(extractName(entity_info))
    result['description'] = translator.get_translation(extractDescription(entity_info))
    #explicitly query using sparql to get main biography data
    for entity, wdt in bio.items():
        spqrqlq = f"SELECT ?entity ?entityLabel ?entityDescription WHERE {{ wd:{wd} wdt:{wdt} ?entity; SERVICE wikibase:label {{ bd:serviceParam wikibase:language \"en\". }} }}"
        v =  ""
        res = get_results(endpoint_url, str(spqrqlq))
        for entities in res['results']['bindings']:
            value = entities.get('entityLabel').get('value', "")
            if value != '' and 'Q' not in value:
                v += value + ','
        if v != "":
            if entity in to_transliteration:
                result[entity] = translator.get_transliteration(v)
            else:
                result[entity] = translator.get_translation(v)
    return result

### Method2

In [16]:
import wptools
import json
from tqdm import tqdm
import re
import requests as r
from collections import defaultdict

In [17]:
def update(infobox , translator):
    infobox = defaultdict(str , infobox)
    updated_infobox = {}
    updated_infobox['name'] = translator.get_transliteration(infobox['name'])
    updated_infobox['native_name'] = infobox['native_name']
    updated_infobox['category'] = translator.get_translation(infobox['settlement_type'])
    
    updated_infobox['image'] = infobox['image_skyline']
    if updated_infobox['image'] == '' : 
        updated_infobox['image'] = infobox['image']
    updated_infobox['image_caption'] = translator.get_translation(infobox['image_caption'])
    updated_infobox['flag'] = infobox['flag']
    updated_infobox['map'] = infobox['map']
    updated_infobox['map_caption'] = translator.get_translation(infobox['map_caption'])
    updated_infobox['motto'] = translator.get_transliteration(infobox['motto'])
    updated_infobox['timezone'] = infobox['timezone']
    
    updated_infobox['country'] = translator.get_translation(infobox['country'])
    updated_infobox['state'] = translator.get_translation(infobox['state'])
    updated_infobox['region'] = translator.get_translation(infobox['region'])
    updated_infobox['district'] = translator.get_translation(infobox['district'])
    updated_infobox['municipality'] = translator.get_translation(infobox['municipality'])
    updated_infobox['location'] = translator.get_translation(infobox['location'])
    updated_infobox['area'] = infobox['area_km2']
    updated_infobox['length'] = infobox['length_km']
    updated_infobox['width'] = infobox['width_km']
    
    updated_infobox['population'] = translator.get_translation(infobox['population'])
    updated_infobox['elevation'] = infobox['elevation_m']
    
    updated_infobox['animal'] = translator.get_translation(infobox['animal'])
    updated_infobox['plant'] = translator.get_translation(infobox['plant'])
    updated_infobox['geology'] = translator.get_translation(infobox['geology'])
    updated_infobox = { key : val for key , val in updated_infobox.items() if val!=''}
    cur_len = len(updated_infobox)
    if len(updated_infobox) < 15:
        for key , val in infobox.items():
            if key in updated_infobox : continue
            updated_infobox[key] = translator.get_transliteration(val)
            if len(updated_infobox) == 15: break
    updated_infobox = { key : val for key , val in updated_infobox.items() if val!=''}
    return updated_infobox

In [18]:
def method2_infobox(name):
    page = wptools.page(name).get_parse()
    result = update(page.data['infobox'] , translator)
    return result

### Baseline

In [23]:
def getEntityInfo(eid):
    return get_entity_dict_from_api(eid)

#extract the name of the entity or the property value in native language
def extractNameBaseline(info):
    return info.get('labels', {}).get('hi', {}).get('value', "")

#extract the description of the entity or the property value in native language
def extractDescriptionBaseline(info):
    return info.get('descriptions', {}).get('hi', {}).get('value', "")

#print the name and the description
def printNameAndDescriptionBaseline(info, trans):
    name = extractName(info)
    if name != "":
        print(trans['name'] + ":", name)
    desc = extractDescription(info)
    if desc != "":
        print(trans['description'] + ":", desc)
    
def baseline_infobox(wd, bio):
    result = {}
    #get entity from api
    entity_info = getEntityInfo(wd)
    #print name and description
#     printNameAndDescription(entity_info, trans)
    result['name'] = extractNameBaseline(entity_info)
    result['description'] = extractDescriptionBaseline(entity_info)
    #explicitly query using sparql to get main biography data
#     print("----------------------",trans['main_info'],"----------------------")
    for entity, wdt in bio.items():
        spqrqlq = f"SELECT ?entity ?entityLabel ?entityDescription WHERE {{ wd:{wd} wdt:{wdt} ?entity; SERVICE wikibase:label {{ bd:serviceParam wikibase:language \"hi\". }} }}"
        v = ""
        res = get_results(endpoint_url, str(spqrqlq))
        for entities in res['results']['bindings']:
            value = entities.get('entityLabel').get('value', "")
            if value != '' and 'Q' not in value:
                v += value + ','
        if v != "":
            result[entity] = v
    return result
#             print(s + v)
            

In [24]:
def get_score(translated_infobox , actual_infobox):
    inst = inexactsearch.InexactSearch()
    markings = defaultdict(list)
    for key in translated_infobox.keys():
        if key in actual_infobox.keys() and key!='image':
#             print(key)
#             print(translated_infobox[key] , actual_infobox[key]) 
            val = inst.compare(translated_infobox[key] , actual_infobox[key])
            if val > 0.70 :
                markings['C'].append(key)
            else : markings['S'].append(key)
    for key in actual_infobox.keys():
        if key not in translated_infobox.keys():
            markings['D'].append(key)
    if len(markings['C']) + len(markings['S']) != 0:
        precision = len(markings['C']) / (len(markings['C']) + len(markings['S']))
    else : precision = 0
    recall = len(markings['C']) / (len(markings['C']) + len(markings['S']) + len(markings['D']))
    print('Precisions :',precision)
    print('Recall :',recall)
    return precision , recall

In [25]:
places_list = [
    ['Narendra Modi', 'नरेन्द्र मोदी', 'Q1058'],
    ['Indira Gandhi', 'इन्दिरा गांधी', 'Q1149'],
    ['Leonard Cohen', 'लिओनार्ड कोहेन', 'Q1276'],
    ['Donald Trump' , 'डॉनल्ड ट्रम्प' , 'Q22686'],
    ['Amitabh Bachchan','अमिताभ बच्चन','Q9570'],
    ['Sachin Tendulkar','सचिन तेंदुलकर','Q9488'],
    ['Virat Kohli','विराट कोहली','Q213854'],
    ['Joe Biden','जो बाइडेन','Q6279'],
    ['Malala Yousafzai' ,'मलाला युसुफ़ज़ई','Q32732']
]

In [26]:
scores_m1 = []
scores_m2 = []
scores_base = []
final_people = []
for en_name , hi_name , qid in places_list:
    print(en_name , hi_name , qid)
    actual_infobox = translator.get_page(hi_name , language = 'hi').data['infobox']
    if actual_infobox == None: continue
    method1 = method1_infobox(qid, properties)
    method2 = method2_infobox(en_name)
    baseline = baseline_infobox(qid,properties)
#     break
    print("For ", en_name)
    print("For method1")
    scores_m1.append(get_score(method1 , actual_infobox))
    print("For method2")
    scores_m2.append(get_score(method2 , actual_infobox))
    print("For baseline")
    scores_base.append(get_score(baseline , actual_infobox))
    if (scores_base[-1][0] > 0.5 or scores_m1[-1][0] > 0.5 or scores_m2[-1][0] > 0.5):
        final_people.append([en_name,hi_name,qid])
        print(final_people[-1])
        #     break

    if len(final_people) == 15: break
print()

Narendra Modi नरेन्द्र मोदी Q1058


hi.wikipedia.org (parse) नरेन्द्र मोदी
hi.wikipedia.org (imageinfo) File:PM Modi Portrait(cropped).jpg
नरेन्द्र मोदी (hi) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:PM Modi P...
  infobox: <dict(26)> image, office, president, term_start, predec...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:N...
  pageid: 26262
  parsetree: <str(111383)> <root><template><title>Infobox officeho...
  requests: <list(2)> parse, imageinfo
  title: नरेन्द्र मोदी
  wikibase: Q1058
  wikidata_url: https://www.wikidata.org/wiki/Q1058
  wikitext: <str(90526)> {{Infobox officeholder|image    = PM Modi...
}
en.wikipedia.org (parse) Narendra Modi
en.wikipedia.org (imageinfo) File:Prime Minister, Shri Narendra M...
Narendra Modi (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Prime Min...
  infobox: <dict(46)> image, image_size, order, office, president,...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:N...
  pageid: 444222
  parsetre

For  Narendra Modi
For method1
Precisions : 0
Recall : 0.0
For method2
Precisions : 0.5
Recall : 0.08
For baseline
Precisions : 0
Recall : 0.0
Indira Gandhi इन्दिरा गांधी Q1149


hi.wikipedia.org (parse) इन्दिरा गांधी
hi.wikipedia.org (imageinfo) File:Indira Gandhi in 1967.jpg
इन्दिरा गांधी (hi) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Indira Ga...
  infobox: <dict(54)> name, image, imagesize, office, president, t...
  pageid: 10189
  parsetree: <str(42896)> <root><template><title>Infobox officehol...
  requests: <list(2)> parse, imageinfo
  title: इन्दिरा गांधी
  wikibase: Q1149
  wikidata_url: https://www.wikidata.org/wiki/Q1149
  wikitext: <str(36301)> {{Infobox officeholder|name     = इन्दिरा...
}
en.wikipedia.org (parse) Indira Gandhi
en.wikipedia.org (imageinfo) File:Indira Gandhi in 1967.jpg
Indira Gandhi (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Indira Ga...
  infobox: <dict(63)> name, image, party, order, office, term_star...
  iwlinks: <list(5)> https://commons.wikimedia.org/wiki/Category:I...
  pageid: 15179
  parsetree: <str(221003)> <root><template><title>short descriptio...
  requests: <list(2)> pa

For  Indira Gandhi
For method1
Precisions : 1.0
Recall : 0.018867924528301886
For method2
Precisions : 0.375
Recall : 0.05660377358490566
For baseline
Precisions : 1.0
Recall : 0.018867924528301886
['Indira Gandhi', 'इन्दिरा गांधी', 'Q1149']
Leonard Cohen लिओनार्ड कोहेन Q1276


...

For  Virat Kohli
For method1
Precisions : 1.0
Recall : 0.01020408163265306
For method2
Precisions : 0.35714285714285715
Recall : 0.05102040816326531
For baseline
Precisions : 1.0
Recall : 0.01020408163265306
['Virat Kohli', 'विराट कोहली', 'Q213854']
Joe Biden जो बाइडेन Q6279


hi.wikipedia.org (parse) जो बाइडेन
hi.wikipedia.org (imageinfo) File:Joe Biden official portrait 2013.jpg
जो बाइडेन (hi) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Joe Biden...
  infobox: <dict(60)> name, image, caption, office, vicepresident,...
  pageid: 777228
  parsetree: <str(11977)> <root><template><title>Current person</t...
  requests: <list(2)> parse, imageinfo
  title: जो बाइडेन
  wikibase: Q6279
  wikidata_url: https://www.wikidata.org/wiki/Q6279
  wikitext: <str(6640)> {{Current person|date=November 2020}}{{Inf...
}
en.wikipedia.org (parse) Joe Biden
en.wikipedia.org (imageinfo) File:Joe Biden official portrait 2013.jpg
Joe Biden (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Joe Biden...
  infobox: <dict(60)> image, caption, office, vicepresident, term_...
  iwlinks: <list(5)> https://commons.wikimedia.org/wiki/Category:J...
  pageid: 145422
  parsetree: <str(332975)> <root><template><title>short descriptio...
  requests: <list(2)

For  Joe Biden
For method1
Precisions : 0.0
Recall : 0.0
For method2
Precisions : 0.46153846153846156
Recall : 0.1016949152542373
For baseline
Precisions : 1.0
Recall : 0.01694915254237288
['Joe Biden', 'जो बाइडेन', 'Q6279']
Malala Yousafzai मलाला युसुफ़ज़ई Q32732


hi.wikipedia.org (parse) मलाला युसुफ़ज़ई


मलाला युसुफ़ज़ई (hi) data
{
  pageid: 497010
  parsetree: <str(21362)> <root><template><title>ज्ञानसन्दूक व्यक्...
  requests: <list(1)> parse
  title: मलाला युसुफ़ज़ई
  wikibase: Q32732
  wikidata_url: https://www.wikidata.org/wiki/Q32732
  wikitext: <str(17798)> {{ज्ञानसन्दूक व्यक्ति|name     = मलाला यु...
}


In [109]:
actual_infobox = translator.get_page('डेविड बोवी' , language = 'hi').data['infobox']
print(actual_infobox)

hi.wikipedia.org (parse) डेविड बोवी


{'Name': 'David Bowie', 'Img': 'Heathen Tour Bowie.jpg', 'Img_capt': 'Bowie during the [[Heathen Tour]] in 2002.', 'Background': 'solo_singer', 'Birth_name': 'David Robert Jones (1947-2016)', 'Born': '{{Birth date |1947|1|8|df|=|yes}} <br />[[Brixton]], London, England - 10 january [[2016]].', 'Years_active': '1964–[[2016]]', 'Occupation': 'Musician, actor, record producer, arranger', 'Voice_type': 'Baritone', 'Instrument': 'Guitar, saxophone, piano, keyboards, synthesizers, [[Mellotron]], [[harmonica]], [[Stylophone]], [[xylophone]], [[vibraphone]], [[Koto (musical instrument)|koto]], drums, percussion', 'Genre': 'Rock, [[glam rock]], [[pop rock]], [[art rock]], [[blue-eyed soul]], [[psychedelic rock]], [[protopunk]]', 'Associated_acts': '[[The Riot Squad]], [[Tin Machine]]', 'Label': '[[Deram Records|Deram]], [[RCA Records|RCA]], [[Rykodisc]], [[Virgin Records|Virgin]], [[EMI Records|EMI]], [[ISO Records|ISO]], [[Columbia Records|Columbia]], [[Bertelsmann Music Group|BMG]], [[Parloph

डेविड बोवी (hi) data
{
  infobox: <dict(14)> Name, Img, Img_capt, Background, Birth_name,...
  iwlinks: <list(4)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 219951
  parsetree: <str(126632)> <root><template><title>Use British Engl...
  requests: <list(1)> parse
  title: डेविड बोवी
  wikibase: Q5383
  wikidata_url: https://www.wikidata.org/wiki/Q5383
  wikitext: <str(103037)> {{Use British English|date=अगस्त 2010}}{...
}


In [101]:
print(scores_base)

[(0.0, 0.0), (0.25, 0.04), (0.25, 0.018867924528301886), (0.3333333333333333, 0.04), (0.5, 0.07142857142857142)]


In [121]:
final_people2 = final_people.copy()
scores_base2 = scores_base.copy()
scores_m1 = scores_m1.copy()
scores_m2 = scores_m2.copy()

In [120]:
scores_m2

[(0.38461538461538464, 0.08620689655172414),
 (0.625, 0.2),
 (0.5454545454545454, 0.11320754716981132),
 (0.3333333333333333, 0.12),
 (0.5, 0.42857142857142855),
 (0, 0.0),
 (0.1111111111111111, 0.1111111111111111),
 (0.1111111111111111, 0.09090909090909091),
 (0, 0.0),
 (0.36363636363636365, 0.26666666666666666),
 (0.3333333333333333, 0.21428571428571427),
 (0.36363636363636365, 0.08333333333333333),
 (0.2857142857142857, 0.18181818181818182),
 (0, 0.0),
 (0.25, 0.08333333333333333)]

In [ ]:
🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦🐦